In [2]:
import boto3
import io
import json
import pandas as pd

from botocore.exceptions import ClientError

AWS_ID   = # AWS KEY ID GOES HERE
AWS_KEY  = # AWS KEY GOES HERE

In [3]:
bucket = "rearc-quest-2024"
folder_path_part1 = "Datasets-Part1/pr_data_0_Current"

# Once again using my personal AWS creds for this
try:
    s3_client = boto3.client('s3', aws_access_key_id = AWS_ID, aws_secret_access_key = AWS_KEY)
    result    = s3_client.list_objects(Bucket = bucket, Prefix = folder_path_part1).get('Contents')[0]
    obj       = s3_client.get_object(Bucket = bucket, Key = result['Key'])
    df_part1  = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8')
    
except ClientError as error:
    print("ERROR WHILE ATTEMPTING TO GET JSON FILE IN S3:", error)

# Fix column names having leading/ending spaces
for column in df_part1.columns:
    df_part1 = df_part1.rename(columns = {column : column.strip()})
# Fix row values having leading/ending spaces
for column, column_dtype in zip(df_part1.columns, df_part1.dtypes):
    if column_dtype == "object":
        df_part1[column] = df_part1[column].apply(lambda x : x.strip() if isinstance(x, str) else x)
df_part1

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
35461,PRS88003203,2022,Q04,115.039,NaN
35462,PRS88003203,2022,Q05,113.713,NaN
35463,PRS88003203,2023,Q01,116.214,NaN
35464,PRS88003203,2023,Q02,116.953,NaN


In [4]:
folder_path_part2 = "Datasets-Part2/JSON"

# Once again using my personal AWS creds for this
try:
    s3_client    = boto3.client('s3', aws_access_key_id = AWS_ID, aws_secret_access_key = AWS_KEY)
    result       = s3_client.list_objects(Bucket = bucket, Prefix = folder_path_part2).get('Contents')[0]
    obj          = s3_client.get_object(Bucket = bucket, Key = result['Key'])
    json_data    = json.loads(obj['Body'].read())

except ClientError as error:
    print("ERROR WHILE ATTEMPTING TO GET JSON FILE IN S3:", error)

df_part2 = pd.json_normalize(json_data['data'])
df_part2

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2021,2021,329725481,united-states
1,01000US,United States,2020,2020,326569308,united-states
2,01000US,United States,2019,2019,324697795,united-states
3,01000US,United States,2018,2018,322903030,united-states
4,01000US,United States,2017,2017,321004407,united-states
5,01000US,United States,2016,2016,318558162,united-states
6,01000US,United States,2015,2015,316515021,united-states
7,01000US,United States,2014,2014,314107084,united-states
8,01000US,United States,2013,2013,311536594,united-states


In [5]:
df_part2_2013_to_2018 = df_part2[(df_part2['ID Year'] >= 2013) & (df_part2['ID Year'] <= 2018)]
mean_and_std_df       = df_part2_2013_to_2018.groupby(['Nation'], as_index=False).agg({'Population' : ['mean', 'std']}).round(2)
mean_and_std_df

Nation   Population            
                         mean         std
0  United States  317437383.0  4257089.54

In [11]:
df_part1_yearly_sum     = df_part1.groupby(['series_id', 'year'], as_index=False)['value'].sum().round(2)
df_best_year_per_series = df_part1_yearly_sum.groupby(['series_id'], as_index=False).agg({'value' : ['max']})
df_best_year_per_series

,series_id,value
,,max
0,PRS30006011,20.50
1,PRS30006012,17.10
2,PRS30006013,704.12
3,PRS30006021,17.60
4,PRS30006022,12.50
...,...,...
277,PRS88003192,282.80
278,PRS88003193,765.82
279,PRS88003201,36.10


In [15]:
# Edit prior DFs to remove unneccessary columns and rename 'ID Year' column to make the merge on possible
df_part1_edit = df_part1[['series_id', 'year', 'period', 'value']]
df_part2_edit = df_part2[['ID Year', 'Population']].rename(columns = {'ID Year' : 'year'})
df_merge      = df_part1_edit.merge(df_part2_edit, how='inner', on='year')
df_merge

,series_id,year,period,value,Population
0,PRS30006011,2013,Q01,1.400,311536594
1,PRS30006011,2013,Q02,0.400,311536594
2,PRS30006011,2013,Q03,0.300,311536594
3,PRS30006011,2013,Q04,0.700,311536594
4,PRS30006011,2013,Q05,0.700,311536594
...,...,...,...,...,...
11065,PRS88003203,2021,Q01,104.239,329725481
11066,PRS88003203,2021,Q02,104.345,329725481
11067,PRS88003203,2021,Q03,106.517,329725481
11068,PRS88003203,2021,Q04,109.157,329725481
